## Loading dataset 

In [2]:
import os 
os.environ['KAGGLE_CONFIG_DIR'] = '/content'

In [3]:
! kaggle datasets download -d rahilmehtaucoe2784/fashion

 91% 173M/191M [00:02<00:00, 58.3MB/s]
100% 191M/191M [00:02<00:00, 68.5MB/s]


In [4]:
%%capture
!unzip \*.zip && rm *.zip &> /dev/null

## installing and importing Libraries 

In [5]:
!pip install -q tflite-model-maker
!pip install -q pycocotools

     |████████████████████████████████| 616 kB 14.3 MB/s 
     |████████████████████████████████| 120 kB 91.5 MB/s 
     |████████████████████████████████| 1.2 MB 67.0 MB/s 
     |████████████████████████████████| 596 kB 66.3 MB/s 
     |████████████████████████████████| 234 kB 78.4 MB/s 
     |████████████████████████████████| 1.1 MB 62.7 MB/s 
     |████████████████████████████████| 87 kB 9.4 MB/s 
     |████████████████████████████████| 3.4 MB 64.4 MB/s 
     |████████████████████████████████| 77 kB 7.6 MB/s 
     |████████████████████████████████| 6.4 MB 69.7 MB/s 
     |████████████████████████████████| 840 kB 41.0 MB/s 
     |████████████████████████████████| 1.1 MB 66.0 MB/s 
     |████████████████████████████████| 25.3 MB 1.5 MB/s 
     |████████████████████████████████| 99 kB 11.6 MB/s 
     |████████████████████████████████| 352 kB 82.3 MB/s 
     |████████████████████████████████| 47.7 MB 1.2 MB/s 
     |████████████████████████████████| 462 kB 72.3 MB/s 
     |█████████████

In [6]:
import numpy as np
import os

#from tflite_model_maker.config import ExportFormat
from tflite_model_maker import model_spec
from tflite_model_maker import object_detector

import tensorflow as tf
assert tf.__version__.startswith('2')

tf.get_logger().setLevel('ERROR')
from absl import logging
logging.set_verbosity(logging.ERROR)
from tensorflow import keras

## Training 

In [7]:
train_image_dir = '/content/Final Img copy 2/train'
train_annotation_dir = '/content/Final Img copy 2/train anotation'
validate_image_dir = '/content/Final Img copy 2/valid'
validate_anaotation_dir = '/content/Final Img copy 2/valid anotation '

In [8]:

spec = model_spec.get('efficientdet_lite3')

In [9]:
train_dir = object_detector.DataLoader.from_pascal_voc(train_image_dir, train_annotation_dir, label_map={1:"Design Kurta", 2: "Plain kurta", 3: "Casual Shirt", 4:"Formal Shirt", 5: "Desgin sherwani", 6:"Plain sherwani", 7:"tshirt"})

In [10]:
validate_dir = object_detector.DataLoader.from_pascal_voc(validate_image_dir, validate_anaotation_dir, label_map={1:"Design Kurta", 2: "Plain kurta", 3: "Casual Shirt", 4:"Formal Shirt", 5: "Desgin sherwani", 6:"Plain sherwani",7:"tshirt"})

In [11]:
model = object_detector.create(train_dir, 
                               model_spec=spec, 
                               epochs=10, 
                               batch_size=5, 
                               train_whole_model=True, 
                               validation_data=validate_dir
                               )

Epoch 1/10
176/176 [==============================] - 151s 542ms/step - det_loss: 1.1178 - cls_loss: 0.8849 - box_loss: 0.0047 - reg_l2_loss: 0.0935 - loss: 1.2113 - learning_rate: 0.0071 - gradient_norm: 2.7085 - val_det_loss: 0.6058 - val_cls_loss: 0.5414 - val_box_loss: 0.0013 - val_reg_l2_loss: 0.0936 - val_loss: 0.6994
Epoch 2/10
176/176 [==============================] - 96s 543ms/step - det_loss: 0.5588 - cls_loss: 0.4914 - box_loss: 0.0013 - reg_l2_loss: 0.0936 - loss: 0.6524 - learning_rate: 0.0058 - gradient_norm: 3.3871 - val_det_loss: 0.5336 - val_cls_loss: 0.5019 - val_box_loss: 6.3318e-04 - val_reg_l2_loss: 0.0937 - val_loss: 0.6273
Epoch 3/10
176/176 [==============================] - 95s 539ms/step - det_loss: 0.4578 - cls_loss: 0.4166 - box_loss: 8.2554e-04 - reg_l2_loss: 0.0937 - loss: 0.5516 - learning_rate: 0.0051 - gradient_norm: 3.0003 - val_det_loss: 0.5428 - val_cls_loss: 0.5246 - val_box_loss: 3.6538e-04 - val_reg_l2_loss: 0.0938 - val_loss: 0.6366
Epoch 4/10
1

## Evaluating the model 

In [12]:
model.evaluate(validate_dir,batch_size=5)

49/49 [==============================] - 17s 256ms/step



{'AP': 0.60313493,
 'AP50': 0.66761446,
 'AP75': 0.65702575,
 'AP_/Casual Shirt': 0.4123898,
 'AP_/Desgin sherwani': 0.6939761,
 'AP_/Design Kurta': 0.4500077,
 'AP_/Formal Shirt': 0.7678286,
 'AP_/Plain kurta': 0.33912784,
 'AP_/Plain sherwani': 0.6880278,
 'AP_/tshirt': 0.8705865,
 'APl': 0.603173,
 'APm': -1.0,
 'APs': -1.0,
 'ARl': 0.879603,
 'ARm': -1.0,
 'ARmax1': 0.8548344,
 'ARmax10': 0.879603,
 'ARmax100': 0.879603,
 'ARs': -1.0}

## Saving Model 

In [13]:
model.export(export_dir='/content/')